<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/TXT_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Flatten, Bidirectional, LSTM, GRU, Dropout, GlobalAveragePooling1D

## Data gathering

In [2]:
(ds_train, ds_val), ds_info = tfds.load("imdb_reviews", split=["train", "test"], shuffle_files=True, as_supervised=True, with_info=True)
ds_info

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL6XS5E/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL6XS5E/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteL6XS5E/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

**Inspect samples from datasets**

Binary classification:
- Negative: 0
- Positive: 1

In [3]:
sample = next(iter(ds_train))
print("Review:", sample[0].numpy())
print("Label:", sample[1].numpy())

Review: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label: 0


## Featurizing

In [4]:
train_reviews, train_labels = [], []
for review, label in ds_train:
  train_reviews.append(str(review.numpy()))
  train_labels.append(label.numpy())

val_reviews, val_labels = [], []
for review, label in ds_val:
  val_reviews.append(str(review.numpy()))
  val_labels.append(label.numpy())

print("Number of training samples:", len(train_reviews))
print("Number of validation samples:", len(val_reviews))

Number of training samples: 25000
Number of validation samples: 25000


**Tokenizing and padding sequences**

The average number of tokens persample isi 233. Hence, we choose `MAX_SEQ_LEN = 256`.

In [5]:
pd.Series(train_reviews).apply(lambda x : len(x.split())).describe()

count    25000.000000
mean       233.776720
std        173.715418
min         10.000000
25%        127.000000
50%        174.000000
75%        284.000000
max       2470.000000
dtype: float64

In [6]:
BATCH_SIZE = 64
VOCAB_SIZE = 8000
EMBED_DIM = 64
MAX_SEQ_LEN = 256

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_reviews)

In [8]:
def create_tfds(tokenizer, X, y):
  seq_X = pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_SEQ_LEN, padding="post", truncating="post")
  return tf.data.Dataset.from_tensor_slices((seq_X, y)).shuffle(1024).batch(BATCH_SIZE)

ds_train = create_tfds(tokenizer, train_reviews, train_labels)
ds_val = create_tfds(tokenizer, val_reviews, val_labels)

## Model Architecture

In [9]:
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, input_length=MAX_SEQ_LEN, output_dim=EMBED_DIM))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 256, 64)           512000    
_________________________________________________________________
dropout (Dropout)            (None, 256, 64)           0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 512,065
Trainable params: 512,065
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

In [11]:
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
391/391 [==============================] - 5s 12ms/step - loss: 0.6534 - acc: 0.7187 - val_loss: 0.6007 - val_acc: 0.7532
Epoch 2/10
391/391 [==============================] - 4s 11ms/step - loss: 0.5318 - acc: 0.8042 - val_loss: 0.4825 - val_acc: 0.8146
Epoch 3/10
391/391 [==============================] - 4s 11ms/step - loss: 0.4213 - acc: 0.8459 - val_loss: 0.4010 - val_acc: 0.8444
Epoch 4/10
391/391 [==============================] - 4s 11ms/step - loss: 0.3523 - acc: 0.8679 - val_loss: 0.3570 - val_acc: 0.8570
Epoch 5/10
391/391 [==============================] - 5s 12ms/step - loss: 0.3123 - acc: 0.8796 - val_loss: 0.3334 - val_acc: 0.8653
Epoch 6/10
391/391 [==============================] - 4s 11ms/step - loss: 0.2875 - acc: 0.8887 - val_loss: 0.3199 - val_acc: 0.8694
Epoch 7/10
391/391 [==============================] - 5s 12ms/step - loss: 0.2690 - acc: 0.8958 - val_loss: 0.3115 - val_acc: 0.8725
Epoch 8/10
391/391 [==============================] - 5s 12ms/step - 

### Transfer learning

In [12]:
ds_train = tf.data.Dataset.from_tensor_slices((train_reviews, train_labels)).batch(BATCH_SIZE)
ds_val = tf.data.Dataset.from_tensor_slices((val_reviews, val_labels)).batch(BATCH_SIZE)

In [13]:
embedding_url = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2"

model = Sequential()
model.add(hub.KerasLayer(embedding_url, input_shape=(), dtype=tf.string, trainable=True))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 48,190,651
Trainable params: 48,190,651
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
391/391 [==============================] - 8s 19ms/step - loss: 0.5072 - acc: 0.7566 - val_loss: 0.3630 - val_acc: 0.8572
Epoch 2/10
391/391 [==============================] - 7s 19ms/step - loss: 0.3074 - acc: 0.8769 - val_loss: 0.2902 - val_acc: 0.8810
Epoch 3/10
391/391 [==============================] - 7s 19ms/step - loss: 0.2510 - acc: 0.9001 - val_loss: 0.2689 - val_acc: 0.8897
Epoch 4/10
391/391 [==============================] - 7s 19ms/step - loss: 0.2159 - acc: 0.9173 - val_loss: 0.2623 - val_acc: 0.8916
Epoch 5/10
391/391 [==============================] - 7s 19ms/step - loss: 0.1964 - acc: 0.9224 - val_loss: 0.2611 - val_acc: 0.8921
Epoch 6/10
391/391 [==============================] - 7s 19ms/step - loss: 0.1783 - acc: 0.9328 - val_loss: 0.2627 - val_acc: 0.8922
Epoch 7/10
391/391 [==============================] - 7s 19ms/step - loss: 0.1652 - acc: 0.9380 - val_loss: 0.2675 - val_acc: 0.8925
Epoch 8/10
391/391 [==============================] - 8s 19ms/step - 